In [2]:
import fitz  # PyMuPDF
import re
from backend import getMeSomeJuicyAnswers

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text




def preprocess_text(text):
    # Remove unwanted characters and extra spaces
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    return text


# Example usage
def getAnswersFromCV(pdf_path):
    cv_text = extract_text_from_pdf(pdf_path)
    preprocessed_text = preprocess_text(cv_text)
    print(preprocessed_text)
    getMeSomeJuicyAnswers(preprocessed_text)
pdf_path = 'cv.pdf'
getAnswersFromCV(pdf_path)

c:\Users\Work\miniconda3\envs\MD\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Work\miniconda3\envs\MD\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).


bernard ambrosio georges computer engineer student faat bernardgeorgesgmailcom faph one famapm arker braga portugal falinke din bernard georges fagithub bernardgeorges skills python java c haskell prolog html css c matlab sql net javascript mongodb nextjs postgres illustrator photoshop interests photography gaming cooking traveling basketball languages portuguese native english advanced c about me i am a highly motivated and ambitious student with a great interest in the area of programming and artiﬁcial intelligence i constantly seek new challenges and adventures i am excited to learn new skills and apply the ones i have i believe that my passion dedication and willingness to learn makes me an ideal candidate for any role education high school escola secundaria carlos amarante z braga portugal bachelors university of minho z braga portugal this institution allowed me to be a part of various projects that i make avail able on my github https githubcombernardgeorges during this projects

Qualifications: ['Engineering']
Locations: []


[]

ner_tag: 
{'O' : 0, 'B-LANG': 1, 'I-LANG': 2, 'B-SKILL': 3, 'I-SKILL': 4}

In [192]:
from datasets import load_dataset

train_ds = load_dataset('csv', data_files='train.csv', split='train')
test_ds = load_dataset('csv', data_files='test.csv', split='train')

print(train_ds)
print(test_ds)

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 512
})
Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 123
})


In [224]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    print(tokenized_inputs)
    word_ids = tokenized_inputs.word_ids(0)
    print(word_ids)  # Map tokens to their respective word.
    label = examples["ner_tags"]
    previous_word_idx = None
    label_ids = []
    i = 0
    for word_idx in word_ids:  # Set the special tokens to -100.
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:  # Only label the first token of a given word.
            i += 1
            label_ids.append(label[i])
        else:
            label_ids.append(-100)
        previous_word_idx = word_idx
    tokenized_inputs["labels"] = label_ids

    return tokenized_inputs

tokenized_train = train_ds.map(
    tokenize_and_align_labels, 
    batched=True,
    remove_columns=train_ds.column_names
)

train_test_split = tokenized_train.train_test_split(test_size=0.2)

print(train_test_split)


c:\Users\Work\miniconda3\envs\MD\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 409
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 103
    })
})


In [233]:
print(train_test_split['train'])

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 409
})


In [242]:
unique_labels = set()
for example in train_test_split['train']['labels']:
    unique_labels.add(example)
print(f"Unique labels in the training set: {unique_labels}")

# Ensure the number of labels matches the dataset
num_labels = len(unique_labels)
print(f"Number of labels: {num_labels}")

Unique labels in the training set: {0, 1, 3, 4, -100}
Number of labels: 5


In [236]:
# Define training arguments
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=4)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_split['train'],
    eval_dataset=train_test_split['test'],
)

# Train the model
trainer.train()

c:\Users\Work\miniconda3\envs\MD\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/78 [00:00<?, ?it/s]

ValueError: not enough values to unpack (expected 2, got 1)

In [243]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

label_names = unique_labels
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", id2label=id2label, label2id=label2id)

c:\Users\Work\miniconda3\envs\MD\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [244]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_split['train'],
    eval_dataset=train_test_split['test'],
)

# Train the model
trainer.train()

  0%|          | 0/78 [00:00<?, ?it/s]

ValueError: not enough values to unpack (expected 2, got 1)